# Overlapping or inside?

This notebook demonstrates how to determine if a sticky note overlaps or is inside of a rectangle (or square) shape.

The sample mural for this notebook is here: [Sample mural](https://github.com/spackows/MURAL-API-Samples/blob/main/murals/sample-02_overlap-inside-rectangle.json)

You can create that mural using the notebook here: [Mural creating notebook](https://github.com/spackows/MURAL-API-Samples/blob/main/notebooks/sample-01_create-a-mural.ipynb)

The sample mural for this notebook looks like the following image:

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-02_overlap-inside-rectangle.png" width="50%" title="Image of a mural" />

## Step 1: Collect the mural ID

You can find the mural ID in the url of a mural.

Mural urls look something like this:

```
https://app.mural.co/t/<workspace>/m/<workspace>/<id>/...
```

What you need to pass to the MURAL API is just after the `/m/`: the \<workspace> and the \<id>.  And you need to join then with a period.

For example, if you have a mural with this url:

```
https://app.mural.co/t/teamideas1234/m/teamideas1234/1234567890123/...
```

Then, the mural ID is: `teamideas1234.1234567890123`

In [35]:
g_mural_id = ""

## Step 2: Collect your OAuth token

In [36]:
g_auth_token = ""

## Step 3: Read the widgets from the mural

### First, do a basic call to see what the result looks like

In [37]:
import requests
import json

def listWidgets( mural_id, auth_token ):
    # https://developers.mural.co/public/reference/getmuralwidgets
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets"
    headers = { "Accept": "application/json", "Authorization": "Bearer " + auth_token }
    response = requests.request( "GET", url, headers = headers )
    response_json = json.loads( response.text )
    return response_json

In [38]:
listWidgets( "dummy", g_auth_token )

{'code': 'MURAL_NOT_FOUND',
 'message': 'The mural was not found or does not exist.'}

In [39]:
listWidgets( g_mural_id, "dummy" )

{'code': 'UNAUTHORIZED',
 'message': "You don't have the required permissions to access this endpoint."}

In [40]:
listWidgets( g_mural_id, g_auth_token )

{'value': [{'contentEditedBy': {'firstName': 'Sarah',
    'lastName': 'Packowski',
    'type': 'member'},
   'contentEditedOn': 0,
   'createdBy': {'firstName': 'Sarah',
    'lastName': 'Packowski',
    'type': 'member'},
   'createdOn': 1650991681040,
   'height': 382.750001220703,
   'hidden': False,
   'hideEditor': False,
   'hideOwner': False,
   'id': '0-1650991681040',
   'instruction': '',
   'locked': False,
   'lockedByFacilitator': False,
   'parentId': None,
   'presentationIndex': -1,
   'rotation': 0,
   'stackingOrder': 0,
   'title': '',
   'updatedBy': {'firstName': 'Sarah',
    'lastName': 'Packowski',
    'type': 'member'},
   'updatedOn': 1650994432929,
   'width': 608.2500024414062,
   'x': 302.94,
   'y': 314.78,
   'shape': 'square',
   'style': {'backgroundColor': '#FFFFFF00',
    'bold': False,
    'font': 'proxima-nova',
    'fontSize': 23,
    'italic': False,
    'strike': False,
    'textAlign': 'center',
    'underline': False,
    'fontColor': '#000000FF'

### Now, add some error handling to make the function more user-friendly

In [41]:
def listWidgets( mural_id, auth_token ):
    # https://developers.mural.co/public/reference/getmuralwidgets
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets"
    headers = { "Accept": "application/json", "Authorization": "Bearer " + auth_token }
    response = requests.request( "GET", url, headers = headers )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( msg )
        return None
    if "value" not in response_json:
        print( "No value returned" )
        return None
    return response_json["value"]

In [42]:
listWidgets( "dummy", g_auth_token )

MURAL_NOT_FOUND The mural was not found or does not exist.


In [43]:
listWidgets( g_mural_id, "dummy" )

UNAUTHORIZED You don't have the required permissions to access this endpoint.


In [44]:
widgets_arr = listWidgets( g_mural_id, g_auth_token )
print( json.dumps( widgets_arr, indent = 2 ) )

[
  {
    "contentEditedBy": {
      "firstName": "Sarah",
      "lastName": "Packowski",
      "type": "member"
    },
    "contentEditedOn": 0,
    "createdBy": {
      "firstName": "Sarah",
      "lastName": "Packowski",
      "type": "member"
    },
    "createdOn": 1650991681040,
    "height": 382.750001220703,
    "hidden": false,
    "hideEditor": false,
    "hideOwner": false,
    "id": "0-1650991681040",
    "instruction": "",
    "locked": false,
    "lockedByFacilitator": false,
    "parentId": null,
    "presentationIndex": -1,
    "rotation": 0,
    "stackingOrder": 0,
    "title": "",
    "updatedBy": {
      "firstName": "Sarah",
      "lastName": "Packowski",
      "type": "member"
    },
    "updatedOn": 1650994432929,
    "width": 608.2500024414062,
    "x": 302.94,
    "y": 314.78,
    "shape": "square",
    "style": {
      "backgroundColor": "#FFFFFF00",
      "bold": false,
      "font": "proxima-nova",
      "fontSize": 23,
      "italic": false,
      "strike": 

## Step 4: Find the stickies and the rectangle

*You can pass an optional parameter to the `/getmuralwidgets` endpoint to collect only widgets of a certain type, but because our mural is so simple, it just as easy to call the endpoint once and then post-process the full list of widgets.

In [45]:
def getStickies( widgets_arr ):
    stickies_arr = []
    for widget in widgets_arr:
        if ( "type" in widget ) and ( "sticky note" == widget["type"] ):
            stickies_arr.append( { "id"     : widget["id"],
                                   "x"      : widget["x"],
                                   "y"      : widget["y"],
                                   "width"  : widget["width"],
                                   "height" : widget["height"],
                                   "text"   : widget["text"]
                                 } )
    return stickies_arr

def getShapes( widgets_arr ):
    shapes_arr = []
    for widget in widgets_arr:
        if ( "type" in widget ) and ( "shape" == widget["type"] ):
            shapes_arr.append( { "id"     : widget["id"],
                                 "x"      : widget["x"],
                                 "y"      : widget["y"],
                                 "width"  : widget["width"],
                                 "height" : widget["height"]
                                } )
    return shapes_arr

In [46]:
stickies_arr = getStickies( widgets_arr )
print( json.dumps( stickies_arr, indent = 2 ) )

[
  {
    "id": "0-1650991688310",
    "x": 233.94,
    "y": 156.77999999999997,
    "width": 138,
    "height": 138,
    "text": "This is outside"
  },
  {
    "id": "0-1650991701198",
    "x": 564.07,
    "y": 239.28,
    "width": 138,
    "height": 138,
    "text": "This overlaps"
  },
  {
    "id": "0-1650991717299",
    "x": 406.07000000000005,
    "y": 397.28,
    "width": 138,
    "height": 138,
    "text": "This is inside"
  }
]


In [47]:
shapes_arr = getShapes( widgets_arr )
print( json.dumps( shapes_arr, indent = 2 ) )

[
  {
    "id": "0-1650991681040",
    "x": 302.94,
    "y": 314.78,
    "width": 608.2500024414062,
    "height": 382.750001220703
  }
]


## Step 5: Figure out if the stickies are overlapping or inside of the rectangle

In [48]:
def stickyInsideOverlapsRectangle( sticky, rectangle ):
    #
    left_edge_overlaps = False
    if ( sticky["x"] >= rectangle["x"] ) and \
       ( sticky["x"] <= ( rectangle["x"] + rectangle["width"] ) ):
        left_edge_overlaps = True
    #
    right_edge_overlaps = False
    if ( ( sticky["x"] + sticky["width"] ) >= rectangle["x"] ) and \
       ( ( sticky["x"] + sticky["width"] ) <= ( rectangle["x"] + rectangle["width"] ) ):
        right_edge_overlaps = True
    #
    top_edge_overlaps = False
    if ( sticky["y"] >= rectangle["y"] ) and \
       ( sticky["y"] <= ( rectangle["y"] + rectangle["height"] ) ):
        top_edge_overlaps = True
    #
    bottom_edge_overlaps = False
    if ( ( sticky["y"] + sticky["height"] ) >= rectangle["y"] ) and \
       ( ( sticky["y"] + sticky["height"] ) <= ( rectangle["y"] + rectangle["height"] ) ):
        bottom_edge_overlaps = True
    
    if left_edge_overlaps and right_edge_overlaps and top_edge_overlaps and bottom_edge_overlaps:
        return "inside"
    
    if ( left_edge_overlaps or right_edge_overlaps ) and ( top_edge_overlaps or bottom_edge_overlaps ):
        return "overlaps"
    
    return "outside"

In [49]:
rectangle = shapes_arr[0];

for sticky in stickies_arr:
    txt = sticky["text"]
    position = stickyInsideOverlapsRectangle( sticky, rectangle )
    print( "[ " + position.ljust(8) + " ]  Sticky text: '" + txt + "'\n" )

[ outside  ]  Sticky text: 'This is outside'

[ overlaps ]  Sticky text: 'This overlaps'

[ inside   ]  Sticky text: 'This is inside'

